In [1]:
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### LIBRERIAS ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Importa la librearia pandas para manejo de los datos
import pandas as pd 
# Importa la libreria ast que ayuda a procesar árboles de la gramática (desanidar)
# Convierte strings a diccionarios
import ast


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### VARIABLES ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
path_data = "./Movies/movies_dataset.csv_transformado.csv"
# Columnas a utilizar por lo que serán integradas en el dataframe
columns_to_use = ['belongs_to_collection','budget','genres','id','original_language',
                    'overview','popularity','production_companies','production_countries',
                    'release_date','revenue','runtime','spoken_languages','status',
                    'title','vote_average','vote_count']
# Lista de columnas numéricas que tienen datos nulos y serán llenados con cero 
columns = ['revenue', 'budget']
# Carga del dataset que contiene la informacion a tranformar, sólo se ocupan ciertas columnas 
df_orig = pd.read_csv(path_data, usecols=columns_to_use, encoding='utf-8', sep=',')
# Se realiza una copia del DataFrame para trabajar en el 
df_transformed = df_orig.copy()
# Variable sobre la que trabaja la desanidación, extrae el nombre o id, para id, presenta problemas
extract = 'name' #'name' or 'id'
# Lista de columnas a separar o desanidar
columns_to_break = ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages']


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### TRANSFORMACIONES DATASET (movies_dataset)###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Ciclo que itera sobre columnas numéricas y llena nulos con ceros, además convierte a tipo int64
for elem in columns:
    df_transformed[elem] = df_transformed[elem].fillna(0)
    # cambia el tipo de dato de la columna budget y  de str a float
    df_transformed[elem] = df_transformed[elem].astype('int64')

#Elimina filas con datos nulos en la columna 'release_date'
df_transformed.dropna(subset=['release_date'], inplace=True)

# Convierte la columna release_date al formato específicado, tipo DateTime 
df_transformed['release_date'] = pd.to_datetime(df_transformed['release_date'])

# Crea la columna release_year que contiene sólo el año de la fecha de estreno
df_transformed['release_year'] = df_transformed['release_date'].dt.year

# Crea la columna 'return' con valores iguales a cero, se divide 'revenue entre budget' en caso de que budget sea cero,
# el valor será cero
df_transformed['return'] = 0
for dato in df_transformed['budget']:
    if (dato != 0):
        df_transformed['return'] = df_transformed['revenue'] / dato
    # else:
    #     df_transformed['return'] = df_transformed['return']

# Transforma la columna a tipo float de 2 decimales
df_transformed['return'] = pd.Series([round(val, 2) for val in df_transformed['return']])
#rellena los valores nulos de la columna return con ceros
df_transformed['return'] = df_transformed['return'].fillna(0)

# Ciclo que itera las columnas a separar y extrae según variable 'id' o 'nombre'
for column in columns_to_break:
    if (df_transformed[column][0][0] == '{'):
        df_transformed[column] = df_transformed[column].fillna('{}')
        if column == 'production_countries' and extract == 'id': 
            extract = 'iso_3166_1'
            df_transformed[column] = df_transformed[column].apply(lambda x: ast.literal_eval(x).get(extract, '') if ast.literal_eval(x) else '{}')
        elif column == 'spoken_languages' and extract == 'id':
            extract = 'iso_639_1'
            df_transformed[column] = df_transformed[column].apply(lambda x: ast.literal_eval(x).get(extract, '') if ast.literal_eval(x) else '{}')
        else:
            extract == 'id'
            df_transformed[column] = df_transformed[column].apply(lambda x: ast.literal_eval(x).get(extract, '') if ast.literal_eval(x) else '{}')
    
    elif (df_transformed[column][0][0] == '['):
        df_transformed[column] = df_transformed[column].fillna('[]')
        if column == 'production_countries' and extract == 'id': 
            extract = 'iso_3166_1'
            df_transformed[column] = df_transformed[column].apply(lambda x: [d[extract] for d in ast.literal_eval(x)] if ast.literal_eval(x) else '[]')
        elif column == 'spoken_languages' and extract == 'id':
            extract = 'iso_639_1'
            df_transformed[column] = df_transformed[column].apply(lambda x: [d[extract] for d in ast.literal_eval(x)] if ast.literal_eval(x) else '[]')
        else:
            extract == 'id'
            df_transformed[column] = df_transformed[column].apply(lambda x: [d[extract] for d in ast.literal_eval(x)] if ast.literal_eval(x) else '[]')


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                        ### TRANSFORMACIONES DATASET (credits)###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                                    ### variables
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
path_data_credits = "./Movies/credits.csv"      # Ruta del csv
df_credits = pd.read_csv(path_data_credits, encoding='utf-8')   # Carga del data set en variable DataFrame
df_credits_reg = df_credits.copy()              # Copia de la variable DataFrame
columns_to_break_credits = ['cast', 'crew']     # Columnas a desanidar
extract = 'name'                                # Valor a extraer


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                                ### FUNCIONES ###
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Función que extrae los nombres de los directores de una línea de texto
def extract_director_names(credits):
    """
        Extrae el nombre de un diccionario en forma de texto, cuando job = Director
        credits: es el diccionario con formato string
    """
    # Convierte el texto a una lista de diccionarios
    try:
        credit_list = ast.literal_eval(credits)
    except (ValueError, SyntaxError):
        return []
    # Extrae los nombres con el trabajo de 'Director'
    directors = [person[extract] for person in credit_list if person['job'] == 'Director']
    return directors

#Ciclo que cambia cada valor de la columna usando la función descrita
for column in columns_to_break_credits:                      # Itera por cada columna que se pretende desanidar
    df_credits[column] = df_credits[column].fillna('[]')     # En caso de que la columna tenga nulos, llenar con []
    if column == 'cast':                                     # Si la columna pertenece a casting, cre actor_names
        df_credits['actor_names'] = df_credits[column].apply(lambda x: [d[extract] for d in ast.literal_eval(x)] if ast.literal_eval(x) else '[]')
    if column == 'crew':                                     # Si la columna es crew, crea director_names, con el nombre del director
        df_credits['director_names'] = df_credits[column].apply(lambda x: extract_director_names(x))

# Elimina las columnas que les extrajimos los datos
df_credits = df_credits.drop(columns=columns_to_break_credits)

# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                ### UNION DE AMBOS DATAFRAMES POR ID
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
# Une los dataframes que fueron generados al limpiar los datos 
df_transformed = df_transformed.merge(df_credits, on='id', how='inner')


# -------------------------------------------------------------------------------------------------------------------------------------------------- #
                                                ### EXPORTAR DF EN FORMATO PARQUET 
# -------------------------------------------------------------------------------------------------------------------------------------------------- #
df_transformed.to_parquet('../Data/data_movies.parquet', compression='gzip')

In [5]:
df_transformed

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,title,vote_average,vote_count,release_year,return,actor_names,director_names
0,Toy Story Collection,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,81.0,[English],Released,Toy Story,7.7,5415,1995,74.71,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,{},65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,104.0,"[English, Français]",Released,Jumanji,6.9,2413,1995,52.56,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]
2,Grumpy Old Men Collection,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,101.0,[English],Released,Grumpier Old Men,6.5,92,1995,0.00,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch]
3,{},16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,127.0,[English],Released,Waiting to Exhale,6.1,34,1995,16.29,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker]
4,Father of the Bride Collection,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,106.0,[English],Released,Father of the Bride Part II,5.7,173,1995,15.32,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45449,{},0,"[Drama, Action, Romance]",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","[Canada, Germany, United Kingdom, United State...",1991-05-13,...,104.0,[English],Released,Robin Hood,5.7,26,1991,0.00,"[Patrick Bergin, Uma Thurman, David Morrissey,...",[John Irvin]
45450,{},0,[Drama],111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,...,360.0,[],Released,Century of Birthing,9.0,3,2011,0.00,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz]
45451,{},0,"[Action, Drama, Thriller]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,...,90.0,[English],Released,Betrayal,3.8,6,2003,0.00,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester]
45452,{},0,[],227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,...,87.0,[],Released,Satan Triumphant,0.0,0,1917,0.00,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov]
